In [1]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.ensemble import IsolationForest
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
df = pd.read_csv('data/listings.csv')
df

/tmp/ipykernel_40307/287009186.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/listings.csv')


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,111501,Sunny Apartment Palermo Boulevard Cerviño,570921,Emilce,NaN,Palermo,-34.579010,-58.412830,Entire home/apt,41089.0,4,155,2024-04-03,1.00,2,314,12,NaN
1,111934,Bright Studio at Palermo Soho,570921,Emilce,NaN,Palermo,-34.581990,-58.427760,Entire home/apt,34847.0,4,131,2024-02-26,0.83,2,355,22,NaN
2,117548,"Palermo, Buenos Aires: Complete flat: ""Alma Mia"".",454822,D. / Lola,NaN,Palermo,-34.572710,-58.418930,Entire home/apt,39340.0,5,5,2024-02-25,0.53,2,210,5,NaN
3,118069,Great 1 Bedroom - 1.5 Bath,530261,Luciano,NaN,Palermo,-34.579780,-58.432230,Entire home/apt,37338.0,14,21,2022-05-31,0.14,29,334,0,NaN
4,11508,Amazing Luxurious Apt-Palermo Soho,42762,Candela,NaN,Palermo,-34.581840,-58.424150,Entire home/apt,59325.0,3,40,2024-03-05,0.28,1,336,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36556,1144078004789481472,Moderno Departamento en Palermo,166538987,Milagro,NaN,Palermo,-34.595630,-58.421773,Entire home/apt,20982.0,14,0,NaN,NaN,1,364,0,NaN
36557,1144079787383088019,"Amplio, moderno y luminoso",218086099,Jorge Gabriel,NaN,Villa Crespo,-34.595043,-58.430225,Entire home/apt,22984.0,1,0,NaN,NaN,1,346,0,NaN
36558,1144089452672163852,Your Home in Palermo Soho,502321403,Lucas,NaN,Villa Crespo,-34.597900,-58.433380,Entire home/apt,26227.0,1,0,NaN,NaN,31,270,0,NaN
36559,1144120524807013364,Moderno Loft para 4 personas en barrio San Telmo,3469227,Federico,NaN,San Telmo,-34.616317,-58.374662,Entire home/apt,48957.0,5,0,NaN,NaN,170,269,0,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36561 entries, 0 to 36560
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              36561 non-null  int64  
 1   name                            36561 non-null  object 
 2   host_id                         36561 non-null  int64  
 3   host_name                       36561 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   36561 non-null  object 
 6   latitude                        36561 non-null  float64
 7   longitude                       36561 non-null  float64
 8   room_type                       36561 non-null  object 
 9   price                           33911 non-null  float64
 10  minimum_nights                  36561 non-null  int64  
 11  number_of_reviews               36561 non-null  int64  
 12  last_review                     

In [4]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,3.656100e+04,3.656100e+04,0.0,36561.000000,36561.000000,3.391100e+04,36561.000000,36561.000000,29679.000000,36561.000000,36561.000000,36561.000000
mean,6.325924e+17,2.032641e+08,NaN,-34.591300,-58.417899,5.954551e+04,5.605454,23.255190,1.455364,18.978830,215.180794,10.225158
std,4.442096e+17,1.929296e+08,NaN,0.018490,0.030271,7.015697e+05,23.071708,39.888967,1.409139,43.477847,117.459621,14.629388
min,1.150800e+04,1.342600e+04,NaN,-34.693700,-58.530890,2.600000e+02,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,4.289224e+07,2.807387e+07,NaN,-34.602477,-58.437889,2.273000e+04,1.000000,1.000000,0.430000,1.000000,113.000000,0.000000
50%,8.212314e+17,1.339204e+08,NaN,-34.590704,-58.419440,3.059800e+04,2.000000,8.000000,1.040000,2.000000,241.000000,4.000000
75%,1.007852e+18,3.947047e+08,NaN,-34.580730,-58.392992,4.371200e+04,4.000000,28.000000,2.090000,13.000000,331.000000,15.000000
max,1.144128e+18,5.744205e+08,NaN,-34.534980,-58.355403,4.371150e+07,1000.000000,813.000000,53.470000,294.000000,365.000000,608.000000


## Remove outliers

In [7]:
top_10_prices = df['price'].nlargest(20)

In [8]:
top_10_prices

26808    43711503.0
2146     39340353.0
2466     39340353.0
13357    39340353.0
13358    39340353.0
23531    39340353.0
7449     37516962.0
26574    31472282.0
34851    31472282.0
28981    24478442.0
32871    24478442.0
9821     22480205.0
33934    21106414.0
12246    16610371.0
34794    13987681.0
30124    13113451.0
21815    11240098.0
31931    10391544.0
7189      8742301.0
15520     8742301.0
Name: price, dtype: float64

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='price',
    y='count()'
)